# Differential expression analyses using limma

In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import scipy as sci
import rpy2
print(rpy2.__version__)
import gseapy as gp
from xlrd import XLRDError
import xlsxwriter
from gprofiler import gprofiler

/Users/sophie/miniconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.9.1


In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()

scanpy==1.0.4+92.g9a754bb.dirty anndata==0.5.10 numpy==1.14.2 scipy==1.0.1 pandas==0.22.0 scikit-learn==0.19.1 statsmodels==0.8.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
xlsxwriter.__version__

'1.0.2'

In [4]:
adata_all = sc.read('./data/adata_processed.h5')

In [5]:
adata_all_hvg = adata_all[:, adata_all.var.highly_variable].copy()

## Differential expression testing approach

We apply the following additional filters to identify differentially expressed genes

1) We consider only genes that are expressed in >10% of cells in either group  
2) DEG with FDR <0.01 including batch and CDR in the design matrix  
3) Filtering DEG based on absolute log2 fold-change >0.5  

## helper functions

In [6]:
def abs_log2fc(adata,grouptype,group2,group1):
    # calculate the absolute log2-foldchange as log2(b/a) = (ln(b)-ln(a))/ln(2)
    # data is ln(counts+1), neglect the +1
    obs_df=pd.DataFrame(adata.X.toarray(),columns=adata.var_names)
    obs_df[grouptype]=adata.obs[grouptype].values    
    x=obs_df.groupby([grouptype])
    tot=x[adata.var_names].apply(np.mean)
    lfc= (tot.loc[group2] - tot.loc[group1])/np.log(2)
    return lfc

In [7]:
def run_limma_counts(adata, condobs, batchobs, countsobs, coef,ref):

    import rpy2.robjects as robjects
    import rpy2.robjects.numpy2ri
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.packages import importr
    pandas2ri.activate()
    
    # prepare data for R
    data=pd.DataFrame(adata.X.toarray(), columns=adata.var_names)
    cond=adata.obs[condobs]
    batch=adata.obs[batchobs]
    counts=adata.obs[countsobs]

    # load R packages and data
    R=robjects.r
    R('library(edgeR)')
    R('library(limma)')
    R.assign('data',data.T)
    #print(dgelist)
    R.assign('cond', cond)
    R.assign('batch', batch)
    R.assign('counts', counts)
    R('counts<-scale(counts)')

    # delete for memory
    del data
    del cond    
    del batch
    
    # format data and create dge object 
    R('cond <- as.factor(cond)')
    R('cond <- relevel(cond,'''+ref+''')''')
    R('dge <- edgeR::DGEList(data,group=cond,sample=data.frame(batch=batch,counts=counts))')
    R('rm(data)')
    R('y <- new("EList")')
    R('y$E <- dge')
    R('rm(dge)')
    
    # design matrix for testing
    R('design <- model.matrix(~cond+batch+counts)')
    
    # run limma
    print('run limma lmFit')
    R('fit <- limma::lmFit(y, design = design)')
    R('rm(y)')
    print('run limma eBayes')
    R('fit <-  limma::eBayes(fit, trend = TRUE, robust = TRUE,)')
    
    # get results
    strg='"BH"'
    tt = R('''limma::topTable(fit,coef='''+coef+''',number = Inf,adjust.method='''+strg+''')''')
    return tt

In [8]:
def filter_amb(adata):
    # ambient genes for filtering, see processing notebook
    ambient_genes=['Itln1','Spink4','Zg16','Lyz1','Defa21','Gm14851','Defa22','Gm15308','Gm15284',
                   'Defa20','Gm15308','Gm14850','Gm7861','Defa17','AY761184', 'Ang4','Agr2','Clps','Tff3','Defa24','Fcgbp']
    ix_amb_genes = np.in1d(adata.var_names,ambient_genes,invert=True)
    return (ix_amb_genes)

In [9]:
def filter_genes(adata, obs_name, group1, group2):
    #### filter genes at least expressed in 10% of cells in either group
    ix = np.isin(adata.obs[obs_name], group1)
    adata_sub = adata[ix].copy()
    filter_1 = sc.pp.filter_genes(adata_sub.X,min_cells=adata_sub.n_obs*0.01, copy=True)
    del adata_sub
    
    ix=np.isin(adata_filt.obs[obs_name], group2)
    adata_sub=adata_filt[ix]
    filter_2=sc.pp.filter_genes(adata_sub.X,min_cells=adata_sub.n_obs*0.01,copy=True)
    del adata_sub
    
    ix_genes=[a or b for a, b in zip(filter_1[0],filter_2[0])]
    
    adata = adata[:,np.array(ix_genes)].copy()
    
    return adata

In [10]:
def scale_lfc(x):
    # split into up-/downregulated genes
    xup=x[x['logFC']>0]
    xdown=x[x['logFC']<0]
    
    # sort by foldchange
    xup.sort_values(by=['logFC'], ascending=True, inplace=True)
    xdown.sort_values(by=['logFC'], ascending=True, inplace=True)    

    # scale upregulated genes
    xup['log_weight']=xup['logFC']
    xup['log_weight'][xup['logFC']>=0.5]=0.6
    xup['log_weight']= (xup['log_weight']-min(xup['log_weight']))/(max(xup['log_weight'])-min(xup['log_weight']))
    
    # scale downregulated genes
    xdown['log_weight']=xdown['logFC']
    xdown['log_weight'][xdown['logFC']<=-0.5]=-0.6
    xdown['log_weight']=abs(xdown['log_weight'])
    xdown['log_weight']= (xdown['log_weight']-min(xdown['log_weight']))/(max(xdown['log_weight'])-min(xdown['log_weight']))

    return xup, xdown

In [11]:
def write_to_excel(df, writer, group, prefix):
    if len(group)>17:
            group_short=group[0:17]
            group_short=group_short.replace('/', '_')
            
            df.to_excel(writer_genes, sheet_name=group_short+prefix)
    else:
            group=group.replace('/', '_')

            df.to_excel(writer_genes, sheet_name=group+prefix)

In [13]:
def run_enrichment(x, writer, group, prefix, database='KEGG_2016'):
    
    # prepare gene list
    x_rank=x['log_weight']
    x_rank=x_rank.reset_index()

    print('run enrichment with enrichR using'+ database)

    #enrichment
    try:        
        enr = gp.enrichr(gene_list=x_rank,
                 description='test_name',
                 gene_sets=database,
                 cutoff=1
                    )
        write_to_excel(enr.res2d, writer, group, prefix)
               
        del enr
    
    except (ValueError,Exception):
        print ('no significant go term..')
        pass

### proximal vs. distal cells in control samples

In [14]:
writer_genes = pd.ExcelWriter('./differential_expression/CD_regionality_genes_final.xlsx', engine='xlsxwriter')

In [15]:
# subset to control cells
adata_all_cd = adata_all[adata_all.obs['diet'].isin(['CD'])].copy()

In [17]:
for i in ['Enterocyte progenitor', 'Enterocyte']:
    print('differential testing in '+ str(i))
    ix_cells = np.isin(adata_all_cd.obs['groups_named_prog4'], [i])  # select cells/cluster for testing
    adata_filt = adata_all_cd[ix_cells,:].copy()
    print('subset  '+ str(np.sum(ix_cells)) + 'cells')
    
    # filter ambient genes
    ix_amb = filter_amb(adata_all) 
    adata_filt = adata_filt[:,ix_amb].copy()
        
    # filter genes at least expressed in 10% of cells in either group
    print('filter genes')
    adata_filt = filter_genes(adata_filt, 'groups_named_regional_only', 'distal', 'proximal')
    
    # run limma
    print('run limma')
    
    try:
        x = run_limma_counts(adata_filt, 'groups_named_regional_only', 'batch', 'n_genes', '"condproximal"', '"distal"')
    
    except rpy2.rinterface.RRuntimeError:
        print ('no significant genes..')
        continue
    
    print('calculate absolute lfc')
    
    # compute absolute log2 fold change
    lfc = abs_log2fc(adata_filt, 'groups_named_regional_only', 'proximal', 'distal')
    x = x.loc[lfc.index]
    x['abs.log2FC'] = lfc.values
    del lfc
    
    # scale lfc to weigh genes for pathway enrichment
    xup, xdown = scale_lfc(x)
    
    # filtering DEG    
    xup = xup[xup['adj.P.Val']<0.01]
    xdown = xdown[xdown['adj.P.Val']<0.01]

    xup = xup[xup['logFC']>0.1]
    xdown = xdown[xdown['logFC']<-0.1]

    # upregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xup), writer_genes, i, '_proximal_up')    
    
    # downregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xdown), writer_genes, i, '_proximal_down')

writer_genes.save()

differential testing in Enterocyte progenitor
subset  3805cells
filter genes
run limma


/Users/sophie/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Lade nötiges Paket: limma

  warnings.warn(x, RRuntimeWarning)


run limma lmFit
run limma eBayes
calculate absolute lfc


/Users/sophie/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/sophie/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sophie/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init

differential testing in Enterocyte
subset  1169cells
filter genes
run limma
run limma lmFit
run limma eBayes
calculate absolute lfc


### CD vs. HFD in major cell types

In [ ]:
writer_genes = pd.ExcelWriter('./differential_expression/Supplementary Table 1.xlsx', engine='xlsxwriter')

In [ ]:
for i in adata_all.obs.groups_named_prog4.cat.categories:
   
    print('differential testing in '+ str(i))
    ix_cells = np.isin(adata_all.obs['groups_named_prog4'], [i])
    adata_filt = adata_all[ix_cells,:].copy()
    print('subset  '+ str(np.sum(ix_cells)) + 'cells')
            
    # filter genes at least expressed in 10% of cells in either group
    print('filter genes')
    adata_filt = filter_genes(adata_filt, 'diet', 'CD', 'HFD')
    
    # run limma
    print('run limma')
    
    try:
        x = run_limma_counts(adata_filt, 'diet', 'batch', 'n_genes', '"condHFD"', '"CD"')
    
    except rpy2.rinterface.RRuntimeError:
        print ('no significant genes..')
        continue
    
    print('calculate absolute lfc')
    
    # compute absolute log2 fold change
    lfc = abs_log2fc(adata_filt, 'diet', 'HFD', 'CD')
    x = x.loc[lfc.index]
    x['abs.log2FC'] = lfc.values
    del lfc
    
    # scale lfc to weigh genes for pathway enrichment
    xup, xdown = scale_lfc(x)
    
    # filtering DEG    
    xup=xup[xup['adj.P.Val']<0.01]
    xdown=xdown[xdown['adj.P.Val']<0.01]

    xup=xup[xup['logFC']>0.1]
    xdown=xdown[xdown['logFC']<-0.1]
    
#     # write all genes to excel genes
#     write_to_excel(pd.DataFrame(x), writer_all, i, '')

    # upregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xup), writer_genes, i, '_up')
        
#     # pathway enrichment
#     run_enrichment(xup, writer, i, '_up', database='KEGG_2016')
    
    
    # downregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xdown), writer_genes, i, '_down')
        
#     # pathway enrichment
#     run_enrichment(xdown, writer, i, '_down', database='KEGG_2016')
    
writer_genes.save()

### CD vs. HFD in cycling cells of major cell types

In [ ]:
writer_genes = pd.ExcelWriter('./differential_expression/Supplementary Table 5.xlsx', engine='xlsxwriter')

In [ ]:
for i in adata_all.obs['groups_named_prog_eec'].cat.categories:
   
    print('differential testing in '+ str(i))
    ix_cells = np.isin(adata_all.obs['groups_named_prog_eec'], [i])
    adata_filt = adata_all[ix_cells,:].copy()
    print('subset  '+ str(np.sum(ix_cells)) + 'cells')
    
    # filter ambient genes
    ix_amb = filter_amb(adata_all) 
    adata_filt = adata_filt[:,ix_amb].copy()

    # subset to cycling cells
    ix_cycle = np.isin(adata_filt.obs['proliferation'],'Cycling')
    adata_filt = adata_filt[ix_cycle].copy()
            
    # filter genes at least expressed in 10% of cells in either group
    print('filter genes')
    adata_filt = filter_genes(adata_filt, 'diet', 'CD', 'HFD')
    
    # run limma
    print('run limma')
    
    try:
        x = run_limma_counts(adata_filt, 'diet', 'batch', 'n_genes', '"condHFD"', '"CD"')
    
    except rpy2.rinterface.RRuntimeError:
        print ('no significant genes..')
        continue
    
    print('calculate absolute lfc')
    
    # compute absolute log2 fold change
    lfc = abs_log2fc(adata_filt, 'diet', 'HFD', 'CD')
    x = x.loc[lfc.index]
    x['abs.log2FC'] = lfc.values
    del lfc
    
    # scale lfc to weigh genes for pathway enrichment
    xup, xdown = scale_lfc(x)
    
    # filtering DEG    
    xup=xup[xup['adj.P.Val']<0.01]
    xdown=xdown[xdown['adj.P.Val']<0.01]

    xup=xup[xup['logFC']>0.1]
    xdown=xdown[xdown['logFC']<-0.1]
    
    # upregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xup), writer_genes, i, '_up')
        
#     # pathway enrichment
#     run_enrichment(xup, writer, i, '_up', database='KEGG_2016')
    
    
    # downregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xdown), writer_genes, i, '_down')
        
#     # pathway enrichment
#     run_enrichment(xdown, writer, i, '_down', database='KEGG_2016')
    
writer_genes.save()

### HFD vs. CD in major cell types split by region

In [ ]:
writer_genes = pd.ExcelWriter('./differential_expression/Supplementary Table 8.xlsx', engine='xlsxwriter')
writer_kegg = pd.ExcelWriter('./differential_expression/Supplementary Table 9.xlsx', engine='xlsxwriter')

In [ ]:
for i in adata_all.obs['groups_named_regional_eec'].cat.categories:
   
    print('differential testing in '+ str(i))
    ix_cells = np.isin(adata_all.obs['groups_named_regional_eec'], [i])
    adata_filt = adata_all[ix_cells,:].copy()
    print('subset  '+ str(np.sum(ix_cells)) + 'cells')
    
    # filter ambient genes
    ix_amb = filter_amb(adata_all) 
    adata_filt = adata_filt[:,ix_amb].copy()
            
    # filter genes at least expressed in 10% of cells in either group
    print('filter genes')
    adata_filt = filter_genes(adata_filt, 'diet', 'CD', 'HFD')
    
    # run limma
    print('run limma')
    
    try:
        x = run_limma_counts(adata_filt, 'diet', 'batch', 'n_genes', '"condHFD"', '"CD"')
    
    except rpy2.rinterface.RRuntimeError:
        print ('no significant genes..')
        continue
    
    print('calculate absolute lfc')
    
    # compute absolute log2 fold change
    lfc = abs_log2fc(adata_filt, 'diet', 'HFD', 'CD')
    x = x.loc[lfc.index]
    x['abs.log2FC'] = lfc.values
    del lfc
    
    # scale lfc to weigh genes for pathway enrichment
    xup, xdown = scale_lfc(x)
    
    # filtering DEG    
    xup=xup[xup['adj.P.Val']<0.01]
    xdown=xdown[xdown['adj.P.Val']<0.01]

    xup=xup[xup['logFC']>0.1]
    xdown=xdown[xdown['logFC']<-0.1]
    
    # upregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xup), writer_genes, i, '_up')
        
    # pathway enrichment
    run_enrichment(xup, writer_kegg, i, '_up', database='KEGG_2016')
    
    # downregulated genes
    # write genes to excel
    write_to_excel(pd.DataFrame(xdown), writer_genes, i, '_down')
        
    # pathway enrichment
    run_enrichment(xdown, writer_kegg, i, '_down', database='KEGG_2016')
    
writer_kegg.save()
writer_genes.save()

### CD vs. HFD in EEC subpopulations

In [ ]:
writer_genes = pd.ExcelWriter('./differential_expression/Supplementary Table 4.xlsx', engine='xlsxwriter')

In [ ]:
for i in ['Sox4+ early EE progenitor', 'Ngn3+ progenitor', 'Arx+/Isl1+ progenitor', 'Pax4+ progenitor', 'Ghrl+ progenitor', 
          'SILA', 'SILP', 'SAKD', 'SIK', 'Reg4+ EC', 'EC', 'Lgr5+ EEC']:
   
    print('differential testing in '+ str(i))
    ix_cells = np.isin(adata_all.obs['groups_named_eec_final'], [i])
    adata_filt = adata_all[ix_cells,:].copy()
    print('subset  '+ str(np.sum(ix_cells)) + 'cells')
    
    # filter ambient genes
    ix_amb = filter_amb(adata_all) 
    adata_filt = adata_filt[:,ix_amb].copy()
            
    # filter genes at least expressed in 10% of cells in either group
    print('filter genes')
    adata_filt = filter_genes(adata_filt, 'diet', 'CD', 'HFD')
    
    # run limma
    print('run limma')
    
    try:
        x = run_limma_counts(adata_filt, 'diet', 'batch', 'n_genes', '"condHFD"', '"CD"')
    
    except rpy2.rinterface.RRuntimeError:
        print ('no significant genes..')
        continue
    
    print('calculate absolute lfc')
    
    # compute absolute log2 fold change
    lfc = abs_log2fc(adata_filt, 'diet', 'HFD', 'CD')
    x = x.loc[lfc.index]
    x['abs.log2FC'] = lfc.values
    del lfc
    
    # write genes to excel
    write_to_excel(pd.DataFrame(x), writer_genes, i, '')    
            
writer_genes.save()